<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



In [101]:
!pip install tensorflow

In [102]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np
from scipy.spatial.distance import cdist
import plotly.graph_objects as go
import plotly.express as px
import os
import platform
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

In [103]:
if platform.system() == 'Windows':
    !curl https://jmp.sh/s/FrvKyzh0WW9GY8kxoDxs -o train.csv
else:
    !wget train.csv https://jmp.sh/s/FrvKyzh0WW9GY8kxoDxs

--2024-11-18 22:42:27--  http://train.csv/
Resolving train.csv (train.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘train.csv’
--2024-11-18 22:42:27--  https://jmp.sh/s/FrvKyzh0WW9GY8kxoDxs
Resolving jmp.sh (jmp.sh)... 52.45.153.116, 44.195.241.53, 54.243.168.71, ...
Connecting to jmp.sh (jmp.sh)|52.45.153.116|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://jumpshare.com/s/FrvKyzh0WW9GY8kxoDxs [following]
--2024-11-18 22:42:27--  https://jumpshare.com/s/FrvKyzh0WW9GY8kxoDxs
Resolving jumpshare.com (jumpshare.com)... 3.93.26.3, 54.243.168.71, 44.195.241.53, ...
Connecting to jumpshare.com (jumpshare.com)|3.93.26.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘FrvKyzh0WW9GY8kxoDxs.1’

FrvKyzh0WW9GY8kxoDx     [ <=>                ] 119.36K  --.-KB/s    in 0.06s   

2024-11-18 22:42:28 (2.06 MB/s) - ‘FrvKyzh0WW9GY8kxoDxs.1’ saved [122223]

FINISHED --202

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [104]:
# Armar el dataset utilizando salto de línea para separar las oraciones/docs
df = pd.read_csv('train.csv')
df = df.drop('Y', axis=1)
df.head()

,text
0,"I don't know man, that Jeep looks really cool ..."
1,Bring back old wits end and lets see how long ...
2,"I don't think he's listening, he's more focuse..."
3,Either that or we are all being tricked by the...
4,You forgot the


In [105]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 707541


### 1 - Preprocesamiento

In [106]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    if isinstance(row[0], str):
        sentence_tokens.append(text_to_word_sequence(row[0]))

<ipython-input-106-7d70ce013eea>:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-106-7d70ce013eea>:8: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [107]:
# Demos un vistazo
sentence_tokens[:1]

[['i',
  "don't",
  'know',
  'man',
  'that',
  'jeep',
  'looks',
  'really',
  'cool',
  'too']]

In [108]:
import statistics
median_length = statistics.median([len(sublist) for sublist in sentence_tokens])
print(median_length)
# Queremos saber la longitud promedia de palabras por documento

9


### Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.

In [109]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [110]:
w2v_model = Word2Vec(
    min_count=20,    # Se usa un valor alto, dada la gran cantidad de data se busca trabajar con las que se usan frecuentemente
    window=4,       # Al ser la media 9 de longitud, se toma 4 para en promedio tomar la mitad
    vector_size=100,       # dimensionalidad de los vectores
    negative=10,    # cantidad de negative samples... 0 es no se usa
    workers=2,      # si tienen más cores pueden cambiar este valor
    sg=0 # Se decide usar CBOW, al mostrar resultados similares mas con un menor tiempo de entrenamiento
)

In [111]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [112]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 707539


In [113]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 15357


### 3 - Entrenar embeddings

In [114]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 7394885.0
Loss after epoch 1: 5879608.0
Loss after epoch 2: 5396131.0
Loss after epoch 3: 4972510.0
Loss after epoch 4: 4956384.0
Loss after epoch 5: 4956238.0
Loss after epoch 6: 4239748.0
Loss after epoch 7: 4142320.0
Loss after epoch 8: 4134856.0
Loss after epoch 9: 4086560.0
Loss after epoch 10: 4060136.0
Loss after epoch 11: 4079392.0
Loss after epoch 12: 3999900.0
Loss after epoch 13: 3957140.0
Loss after epoch 14: 1230088.0
Loss after epoch 15: 455112.0
Loss after epoch 16: 422576.0
Loss after epoch 17: 396912.0
Loss after epoch 18: 362368.0
Loss after epoch 19: 327400.0


(110501900, 148662700)

### Probar términos de interés y explicar similitudes en el espacio de embeddings.

*   Bird (Positivo)
*   Animal  (Positivo)
*   Cat (Negativo)
*   Dog (Negativo)



In [115]:
w2v_model.wv.most_similar(positive=["bird"], topn=10)
# crab, lion, tiger, crocodile son animales al igual que bird, aunque depredadores del mismo, posiblemente alli este su relacion
# dome , puede estar asocia al lugar donde suele estar las aves en las ciudades

[('crab', 0.6041927337646484),
 ('rod', 0.5905865430831909),
 ('lion', 0.5628712177276611),
 ('tiger', 0.5614820718765259),
 ('lady', 0.54254549741745),
 ('freddy', 0.5419260263442993),
 ('crocodile', 0.533617377281189),
 ('dome', 0.5331794023513794),
 ('stuffed', 0.5298871994018555),
 ('jacket', 0.5264739394187927)]

In [128]:
w2v_model.wv.most_similar(positive=["animal"], topn=10)
# abomination, ambulance, object No es claro cual podria ser su relacion
# animals un plural
# exotic puede referirse tipo de animales
# species distintos tipos de animales
# elephant un animal

[('rider', 0.4955395460128784),
 ('cobra', 0.4923752248287201),
 ('ghoul', 0.49218985438346863),
 ('terra', 0.470257431268692),
 ('mole', 0.46943578124046326),
 ("king's", 0.4682244658470154),
 ('mountain', 0.4673154354095459),
 ('worm', 0.4669380784034729),
 ('infested', 0.459643691778183),
 ('lion', 0.45896008610725403)]

In [133]:
w2v_model.wv.most_similar(negative=["cat"], topn=10)
# No se ve relacion alguna con cat, lo cual es esperado dado que se busca sus negativos

[("israel's", 0.35738447308540344),
 ('todays', 0.35454437136650085),
 ('agreements', 0.3384774923324585),
 ('encourages', 0.33042624592781067),
 ('provides', 0.3208719789981842),
 ('supported', 0.319030225276947),
 ('arming', 0.3172754645347595),
 ('overseas', 0.3170539140701294),
 ('suspend', 0.3156990706920624),
 ('sanctions', 0.3135536015033722)]

In [136]:
w2v_model.wv.most_similar(negative=["dog"], topn=10)
# No se ve relacion alguna con dog, lo cual es esperado dado que se busca sus negativos

[('provides', 0.33657166361808777),
 ('releases', 0.33413073420524597),
 ('predicted', 0.31953728199005127),
 ('supported', 0.31746944785118103),
 ('bound', 0.3173179626464844),
 ('contributed', 0.3118473291397095),
 ('reports', 0.2954206168651581),
 ('requires', 0.2947520613670349),
 ('confirmed', 0.29360663890838623),
 ('encourages', 0.2928029000759125)]

### Graficar los embeddings resultantes.

In [119]:
def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [120]:
def filter_close_words(word, model, vectors, labels, max_words=50):

    target_vector = model.wv[word].reshape(1, -1)
    distances = cdist(target_vector, model.wv.vectors, metric='cosine').flatten()

    close_indices = np.argsort(distances)[:max_words]
    filtered_vectors = vectors[close_indices]
    filtered_labels = labels[close_indices]

    return filtered_vectors, filtered_labels

In [121]:
vecs_2d, labels_2d = reduce_dimensions(w2v_model)

In [124]:
word_of_interest = "animal"
filtered_vecs, filtered_labels = filter_close_words(word_of_interest, w2v_model, vecs_2d, labels_2d, 10)

fig = px.scatter(x=filtered_vecs[:, 0], y=filtered_vecs[:, 1], text=filtered_labels)
fig.update_traces(textposition='top center')
fig.update_layout(title=f'Words Close to "{word_of_interest}"', xaxis_title="X", yaxis_title="Y")
fig.show(renderer="colab")

### Intentar plantear y probar tests de analogías.

In [137]:
word1 = "domestic"
word2 = "dog"
word3 = "wild"

result = w2v_model.wv.most_similar(positive=[word2, word3], negative=[word1])
print(f"'{word1}' es a '{word2}' como '{word3}' es a '{result[0][0]}'")

'domestic' es a 'dog' como 'wild' es a 'cat'


### Conclusiones

*   Dado lo grande del dataset, se vio que se puede reducir el tamaño de los embedding sin afectar el rendimiento
*   CBOW es mas rapido en entrenar que skip graw
*   Si bien el espacio en el que se encuentra una palabra puede contener palabras asociadas, tambien puede haber palabras que al menos  a primera vista no parece tener relacion, esto puede set, dado el dataset dificil determinar si corresponde o se solapan contexto en el espacio de contextos, ejemplo, para animal alchool esta muy cerca, mientras que su pluran animal mas lejos



